In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [13]:

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage

PERSIST_DIR = './storage'
if not os.path.exists(PERSIST_DIR):
    # load documents and create index
    documents = SimpleDirectoryReader("Data").load_data()
    index = VectorStoreIndex.from_documents(documents, show_progress = True)
    # store it for consecutive use
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

c:\Users\Admin\miniconda3\envs\env-01\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 3/3 [00:01<00:00,  1.98it/s]


In [75]:
from llama_index.core.chat_engine import ContextChatEngine
from llama_index_client import ChatMessage, MessageRole

from llama_index.core.memory import ChatMemoryBuffer

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

# create index with Documents + LLM
index2 = VectorStoreIndex.from_documents(
    documents, embed_model=Settings.embed_model, llm=Settings.llm
)
chatRetreiver = index2.as_retriever()
memory = ChatMemoryBuffer.from_defaults(token_limit=5500)

messageHistory =  [ 
                    ChatMessage(
                        role=MessageRole.SYSTEM,
                        content=
                                "You are an expert assitant for extracting insights from resume in PDF format. \n"
                                "You extract data and return it in JSON format, as below \n"
                                "  {\n"
                                "    CompanyName1:\n"
                                "    JobTitle:\n"
                                "    JobPeriod:\n"
                                "  }\n"
                                "REMEMBER to return extracted data only from provided resume.",
                        additional_kwargs={"2":"2"}
                        ),
                    ChatMessage(
                        role=MessageRole.USER,
                        content="Hello assistant, we are having a insightful discussion about Ramya Anand today.",
                        additional_kwargs={"2":"2"}
                        ),
                    ChatMessage(
                        role=MessageRole.ASSISTANT, 
                        content="Okay, sounds good.",
                        additional_kwargs={"2":"2"}
                        ),
                  ]

newMessage = "List the companies Ramya Anand has worked, her role, job period in Json format?"

chatEngine = ContextChatEngine(chatRetreiver, Settings.llm, memory= memory, prefix_messages= messageHistory)
response3 = chatEngine.chat(newMessage)
print(response3)


{
    "CompanyName1": "Modaxo (Constellation Software, Canada)",
    "JobTitle": "Full stack Developer",
    "JobPeriod": "06/2021 - 04/2024"
},
{
    "CompanyName2": "Virtusa (CIBC - Capital Market, Canada)",
    "JobTitle": "Senior Application Developer",
    "JobPeriod": "02/2020 - 06/2021"
}


In [67]:
response4 = chatEngine.chat("What are the other companies she worked for apart from the ones listed already? \n"
                            "Return the response in same Json format, continue with the company serial numbers")
print(response4)

{
    "CompanyName3": "Bank of NewYork Mellon, US / India",
    "JobTitle": "Lead - Application Development",
    "JobPeriod": "07/2017 - 12/2019"
  },
  {
    "CompanyName4": "iNautix Technologies, US / India",
    "JobTitle": "Technical Lead",
    "JobPeriod": "07/2014 - 06/2017"
  },
  {
    "CompanyName5": "iNautix Technologies, US / India",
    "JobTitle": "Senior Application Developer",
    "JobPeriod": "04/2012 - 06/2014"
  },
  {
    "CompanyName6": "iNautix Technologies, US / India",
    "JobTitle": "Application Developer",
    "JobPeriod": "07/2009 - 03/2012"
  }
}
